## Explicit Euler vs. Implicit-in-velocity Euler
Explicit Euler and implicit-in-velcoity Euler are both used to update the state of a system. However, they work a little bit differently. Explicit Euler updates the state based on the the current state of the system as you can see from the equation:
$$
y_{n+1} = y_n + \Delta t \cdot f(t_n, y_n)
$$
Where:
- $y_{n+1}$ is the state at the next time step
- $y_n$ is the current state at the current time step
- $\Delta t \cdot$ is the time step
- $f(t_n, y_n)$ is the function that describes the system's dynamics at the current time step

While, implicit-in-velocity Euler updates the velocity implicity and the positions explicitly. The equation for velocity is:
$$
v_{t+h} = v_t + h * a(v_{t+h})
$$
Where:
- $v_{t+h}$ is the velocity at the next time step (this is unknown and has to be solved for)
- $v_t$ is the current velocity
- $h$ is the time step
- $a(v_{t+h})$ is the acceleration at the next time step, which depends on the velocity at the next time step

Since $a(v_{t+h})$ is a function of $v_{t+h}$ this forms a non-linear equation. In order to solve for $v_{t+h}$ Mujoco uses a **first-order Taylor Expansion** around $v_t$ since we cannot evaluate $a(v_{t+h})$ without the next velocity. This takes the form of: 
$$
a(v_{t+h}) \approx a(v_t) + \frac{\partial v}{\partial a(v)} \cdot (v_{t+h} - v_t)
$$
Where:
- $a(v_{t+h})$ repesents the acceleration at time $t+h$ and is a function of velocity at the next time step
- $a(v_t)$ is the acceleration at the current time and is a function of the current velocity
- $\frac{\partial v}{\partial a(v)} \cdot$ is the derivative of velocity with respect to acceleration, which is used to approximate the acceleration at the next time step
- $(v_{t+h} - v_t)$ is the change in velocity over the next time step

The derivative is given by:
$$
\frac{\partial v}{\partial a(v)} = M^{-1} D
$$
Where: 
- $M^{-1}$ is the inverse of the mass matrix for the system. $M$ represents the distribution of mass in the system and how it relates to forces acting on the objects
- $D$ is the damping matrix that encodes how the forces change with velocities, excluding constraint forces. 

This simplifies to the following equation to give us the velocity update: 
$$
v_{t+h} = v_t + h \hat{M}^{-1} M a(v_t)
$$
With:
$$
\hat{M} \equiv M - hD
$$

The position is then updated using $v_{t+h}$:
$$
q_{t+h} = q_t + h * v_{t+h}
$$

The main similarities between these two algorithms are that they both predict the state of the system at the next time step based on the current conditions, and they are both first-order integration methods meaning they use the first derivative or the rate of change of the system's state to predict the next state. The main difference is that one is explicit and the other is implicit. Explicit means that it computes the next state of the system using values that have already been computed, while implicit uses values that depend on the unknown future state. Another difference is the explicit Euler is stable for smaller time steps, but can become unstable if the time step is too large. The implicit-in-velocity Euler tends to be more stable for stiff systems since it computes the velocity from the next time step. The explicit Euler is also relatively simple to implement, as we did in class, while the implicit-in-velocity Euler requires solving a non-linear equation for the velocity update which is much more difficult. Overall, both algorithms update the state of the system, explicit Euler is easier to implement, but less stable, and the implicit-in-velovity Euler is more complex, but more stable especially for systems with damping, springs, and/or stiff systems.



In [61]:
import mujoco
import mediapy as media
import numpy as np

# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("trampoline.xml")  # Ensure the file path is correct
data = mujoco.MjData(model)

# Define simulation parameters
duration = 5.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)


This browser does not support the video tag.


In [103]:
# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("trampoline2.xml")
data = mujoco.MjData(model)

# Set initial velocity for the ball (assuming it's the first body)
data.qvel[0:3] = [0, 0, -10]  # Set linear velocity in x, y, z directions

# After loading the model
model.opt.gravity[2] = -9.81  # Reduce gravity (default is -9.81)

# Define simulation parameters
duration = 10.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


In [149]:
import mujoco
import mediapy as media
import numpy as np

# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("tramphumanoid.xml")  # Ensure the file path is correct
data = mujoco.MjData(model)

model.opt.gravity[2] = -3  # Reduce gravity (default is -9.81)
# Define simulation parameters
duration = 8.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)

This browser does not support the video tag.
